In [ ]:
import numpy as np
import pickle
import glob
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import zipfile

In [ ]:
!unzip /content/datasetGYMVision.zip -d /content/datasetV2

Archive:  /content/datasetGYMVision.zip
   creating: /content/datasetV2/dataset/benchPress/
  inflating: /content/datasetV2/dataset/benchPress/benchPress_0.jpg  
  inflating: /content/datasetV2/dataset/benchPress/benchPress_1.jpg  
  inflating: /content/datasetV2/dataset/benchPress/benchPress_10.jpg  
  inflating: /content/datasetV2/dataset/benchPress/benchPress_11.jpg  
  inflating: /content/datasetV2/dataset/benchPress/benchPress_12.jpg  
  inflating: /content/datasetV2/dataset/benchPress/benchPress_13.jpg  
  inflating: /content/datasetV2/dataset/benchPress/benchPress_14.jpg  
  inflating: /content/datasetV2/dataset/benchPress/benchPress_15.jpg  
  inflating: /content/datasetV2/dataset/benchPress/benchPress_16.jpg  
  inflating: /content/datasetV2/dataset/benchPress/benchPress_17.jpg  
  inflating: /content/datasetV2/dataset/benchPress/benchPress_18.jpg  
  inflating: /content/datasetV2/dataset/benchPress/benchPress_19.jpg  
  inflating: /content/datasetV2/dataset/benchPress/benchPr

In [ ]:
#import datasetnya (ImageDataGenerator)( augmentasi juga)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
TRAINING_DIR = "/content/datasetV2/dataset"

training_datagen = ImageDataGenerator(
        rescale=1 / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2  # Set validation split
    )

train_generator = training_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(299, 299),
        class_mode='categorical',
        subset='training'  # Set as training data
    )

validation_generator = training_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(299, 299),
        class_mode='categorical',
        subset='validation'  # Set as validation data
    )

Found 657 images belonging to 8 classes.
Found 163 images belonging to 8 classes.


In [ ]:
print(train_generator.class_indices)

{'Elliptical Machine': 0, 'Home Machine': 1, 'Recumbent Bike': 2, 'benchPress': 3, 'dumbBell': 4, 'kettleBell': 5, 'pullBar': 6, 'treadMill': 7}


In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

xtn = Xception(
    include_top=True,
    weights='imagenet',
    classes=1000,
    classifier_activation="softmax"
)

xtn.trainable = False

model = Sequential([
    xtn,
    Flatten(),
    Dense(200, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(100, activation='relu'),
    Dropout(0.3),
    Dense(50, activation='relu'),
# #jumlah kelas sesuai dengan jumlah alat
    Dense(8, activation='softmax')
])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 1000)              22910480  
                                                                 
 flatten_1 (Flatten)         (None, 1000)              0         
                                                                 
 dense_4 (Dense)             (None, 200)               200200    
                                                                 
 batch_normalization_9 (Bat  (None, 200)               800       
 chNormalization)                                                
                                                                 
 dropout_2 (Dropout)         (None, 200)               0         
                                                                 
 dense_5 (Dense)             (None, 100)               20100     
                                                      

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = Adam(learning_rate=0.001), metrics = ['accuracy'])


history = model.fit(train_generator, epochs=50, validation_data=validation_generator)

Epoch 1/50
21/21 [==============================] - 30s 1s/step - loss: 0.4083 - accuracy: 0.8752 - val_loss: 0.4264 - val_accuracy: 0.8712
Epoch 2/50
21/21 [==============================] - 24s 1s/step - loss: 0.3652 - accuracy: 0.8935 - val_loss: 0.3694 - val_accuracy: 0.8834
Epoch 3/50
21/21 [==============================] - 24s 1s/step - loss: 0.3026 - accuracy: 0.9056 - val_loss: 0.4157 - val_accuracy: 0.8896
Epoch 4/50
21/21 [==============================] - 26s 1s/step - loss: 0.2998 - accuracy: 0.9011 - val_loss: 0.3662 - val_accuracy: 0.8896
Epoch 5/50
21/21 [==============================] - 25s 1s/step - loss: 0.3010 - accuracy: 0.8980 - val_loss: 0.3619 - val_accuracy: 0.8589
Epoch 6/50
21/21 [==============================] - 25s 1s/step - loss: 0.3112 - accuracy: 0.8889 - val_loss: 0.3349 - val_accuracy: 0.9018
Epoch 7/50
21/21 [==============================] - 24s 1s/step - loss: 0.3290 - accuracy: 0.9011 - val_loss: 0.3211 - val_accuracy: 0.8957
Epoch 8/50
21/21 [==

In [ ]:
# Simpan model ke dalam format .h5
model.save("gymvision_modelV2.h5")

# Beritahu pengguna bahwa model telah disimpan
print("Model telah disimpan sebagai 'gymvision_modelV2.h5'")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model telah disimpan sebagai 'gymvision_modelV2.h5'


In [ ]:
from google.colab import files
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the pre-trained model
model = load_model('gymvision_modelV2.h5')  # Ganti dengan nama dan path model Anda

# Function to handle file upload and perform prediction
def handle_upload_and_predict(uploaded_file):
    img = image.load_img(uploaded_file, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    # Perform prediction
    prediction = model.predict(img_array)

    # Convert prediction to class name
    class_names = ['Elliptical Machine2', 'Home Machine2', 'Recumbent Bike2','Benchpress2','dumbell2', 'kettleBell2','Pull Bar2', 'treadMill2']
    predicted_class = class_names[np.argmax(prediction)]

    print("Predicted class:", predicted_class)

# Display upload button
upload_button = files.upload()

# Handle file upload and perform prediction
for filename in upload_button.keys():
    handle_upload_and_predict(filename)


Saving benchPress_52.jpg to benchPress_52.jpg
Saving benchPress_54.jpg to benchPress_54.jpg
Saving benchPress_66.jpg to benchPress_66.jpg
Saving benchPress_74.jpg to benchPress_74.jpg
1/1 [==============================] - 1s 726ms/step
Predicted class: Benchpress2
1/1 [==============================] - 0s 24ms/step
Predicted class: Benchpress2
1/1 [==============================] - 0s 22ms/step
Predicted class: Benchpress2
1/1 [==============================] - 0s 24ms/step
Predicted class: Benchpress2
